TREINAMENTO DO MODELO DE DETECÇÃO DE LETRAS EM LIBAS

In [3]:
# IMPORT
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import MaxPooling3D
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
# DATASET USADO PARA ESSE TESTE: https://www.kaggle.com/datasets/deeppythonist/american-sign-language-dataset?resource=download
base_dir = 'ASL_Gestures_36_Classes' # Definir o diretório onde estão os dados train e test. No nosso caso, será o do grupo e o coletivo da sala.
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

image_size = (32, 32) # Testei 64x64, mas ele pegava muito detalhe, então reduzi para 32x32 (ficou melhor, mas ainda precisa ficar procurando um ângulo bom para as imagens no ao vivo)

batch_size = 32
num_classes = 36 # Esse é o número para o ASL, mas no nosso caso, provavelmente vamos mudar para 26 (alfabeto)

# O dataset deve estar no formato:
# pasta_geral_dataset/
#     train/a,b,c,etc
#     test/a,b,c,etc
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

print("Carregando dados de TREINO...")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    seed=42
)

# Para usar no validate
print(f"\nClasses detectadas (treino): {train_generator.class_indices}")
class_names = list(train_generator.class_indices.keys())
print(f"\nTotal de classes: {len(class_names)}")
print(f"Ordem das classes: {class_names}")

Carregando dados de TREINO...
Found 2011 images belonging to 36 classes.

Classes detectadas (treino): {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33, 'y': 34, 'z': 35}

Total de classes: 36
Ordem das classes: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
print("\nCarregando dados de TESTE...")
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

print(f"\nClasses detectadas (teste): {test_generator.class_indices}")
print(f"Total de classes no teste: {len(test_generator.class_indices)}")


Carregando dados de TESTE...
Found 504 images belonging to 36 classes.

Classes detectadas (teste): {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33, 'y': 34, 'z': 35}
Total de classes no teste: 36


In [ ]:
# LAYERS
img_height, img_width = image_size[0], image_size[1]
input_shape = (img_height, img_width, 3) # 3 para imagens coloridas (RGB)

model = keras.Sequential([ # Esse modelo está funcionando, talvez não precise de mudanças, mas dá para testar outras se der tempo
    # CNN
    layers.Conv2D(
        filters=32, 
        kernel_size=(3,3),
        activation='relu',
        input_shape=input_shape,
        padding='same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(
        filters=64,
        kernel_size=(3,3),
        activation='relu',
        padding='same'),
    layers.MaxPooling2D((2,2)),
    
    # Flatten
    layers.Flatten(),
    
    # Camadas densas
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Evita overfitting por algum motivo
    layers.Dense(num_classes, activation='softmax')
])

model.summary()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

c:\Users\icaro\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 36)             │         9,252 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,077,476 (4.11 MB)

 Trainable params: 1,077,476 (4.11 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping
# Evita overfitting também - se o modelo não melhorar (val_loss) por 5 épocas (patience), para o treinamento e pega os melhores pesos
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


history = model.fit(train_generator, epochs=100, verbose=1, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/100


c:\Users\icaro\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.0983 - loss: 3.4128 - val_accuracy: 0.5694 - val_loss: 1.5968
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.3512 - loss: 2.1852 - val_accuracy: 0.7222 - val_loss: 0.8630
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.4747 - loss: 1.6364 - val_accuracy: 0.8393 - val_loss: 0.5363
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.5682 - loss: 1.2936 - val_accuracy: 0.8591 - val_loss: 0.4533
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.6121 - loss: 1.1306 - val_accuracy: 0.8909 - val_loss: 0.3586
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.6538 - loss: 1.0251 - val_accuracy: 0.9067 - val_loss: 0.3329
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.7142 - loss: 0.8365 - val_accuracy: 0.9028 - val_loss: 0.3070
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.7286 - loss: 0.8132 - val_accuracy: 0.9306 - val_l

In [8]:
model.save('asl_gesture_model2.h5')